In [277]:
import pandas as pd 
from fuzzyfinder import fuzzyfinder as fz
from unidecode import unidecode

In [278]:
file_1 = pd.read_csv('./new_etablissement.csv')
file_2 = pd.read_csv('./Destinations.csv')

In [279]:
file_1.head()

,id,lieu_id,nom_etablissement,adresse
0,1,512,RESIDENCE LE MANUJO,LANSLEVILLARD
1,2,387,RESIDENCE LES ETERLOUS,"73480 Bonneval-sur-Arc, France"
2,3,512,CENTRE EDELWEISS,"2 Rue du Mont-Cenis, 73480 Val-Cenis, France"
3,4,512,CLUB MMV LE VAL CENIS,"Quartier des Sablons, Chem. des Crueux, 73480 ..."
4,6,512,ARBA NEIGE,"322 Rue des Essarts, 73480 Val-Cenis, France"


In [280]:
file_2.head()

,code,id_g2a,name,name_g2a,locality,station
0,NaN,1733.0,Hôtel La Grange d'Arly,HOTEL LA GRANGE D'ARLY,Megève,MEGEVE - DEMI QUARTIER
1,NaN,5962.0,Hôtel Du Midi,HOTEL DU MIDI,Thônes,NaN
2,NaN,6161.0,Hotel Christiania,HOTEL CHRISTIANIA,La Clusaz,NaN
3,NaN,NaN,Le Carmélite,LE CARMELITE,Poitiers,NaN
4,NaN,NaN,Le Calgary,LE CALGARY,Les Saisies,NaN


In [281]:
name_corres = []
for x in range(len(file_1)):
    sub = file_1.loc[x].to_dict()
    sub['loc'] = x
    name_corres.append(sub)

In [282]:
name_corres

[{'id': 1,
  'lieu_id': 512,
  'nom_etablissement': 'RESIDENCE LE MANUJO',
  'adresse': 'LANSLEVILLARD',
  'loc': 0},
 {'id': 2,
  'lieu_id': 387,
  'nom_etablissement': 'RESIDENCE LES ETERLOUS',
  'adresse': '73480 Bonneval-sur-Arc, France',
  'loc': 1},
 {'id': 3,
  'lieu_id': 512,
  'nom_etablissement': 'CENTRE EDELWEISS',
  'adresse': '2 Rue du Mont-Cenis, 73480 Val-Cenis, France',
  'loc': 2},
 {'id': 4,
  'lieu_id': 512,
  'nom_etablissement': 'CLUB MMV LE VAL CENIS',
  'adresse': 'Quartier des Sablons, Chem. des Crueux, 73480 Val-Cenis, France',
  'loc': 3},
 {'id': 6,
  'lieu_id': 512,
  'nom_etablissement': 'ARBA NEIGE',
  'adresse': '322 Rue des Essarts, 73480 Val-Cenis, France',
  'loc': 4},
 {'id': 8,
  'lieu_id': 16,
  'nom_etablissement': 'PARKING DU COL',
  'adresse': '73620 Hauteluce, France',
  'loc': 5},
 {'id': 9,
  'lieu_id': 16,
  'nom_etablissement': 'PARKING DE LA FORET',
  'adresse': '1830 Rte du Mont Bisanne, 73270 Villard-sur-Doron, France',
  'loc': 6},
 {'id

In [283]:
def suggest_simillar_word(item_to_find:dict, list_items:list) -> list:
    from unidecode import unidecode
    word_to_find = unidecode(item_to_find['name']).upper()
    first_suggestions = []
    suggestions = []
    reserved_words = ["LES", "LE", "LA", "DES", "DE", "DU", "A",
                    "RESIDENCE", "RESIDENCES", "RÉSIDENCE", "CENTRE", "CLUB", "HOTEL",
                    "PARKING", "SPI", "&", "AGENCE", "/", "REFUGE",
                    "HÔTEL", "GITE", "GITES","GÎTE", "PARTICULIER", "MVA", "P&V", "VACANCEOLE", "CITADINES",
                    "RESIDETUDES", "CITY", "AIRE", "VVF", "VCS"]
    words = word_to_find.split(' ')
    words = [x for x in words if x not in reserved_words]
    for word in words:
        for item in list_items:
            name_item = str(item['nom_etablissement']).upper()
            similar = fz(word, name_item.split(' '))
            if list(similar):
                if len(name_item) <= len(word_to_find) + 20:
                    if item not in first_suggestions:
                        first_suggestions.append(item)

    for suggestion in first_suggestions:
        splitted = item_to_find['name'].split(' ')
        number_of_word = len(splitted)
        correspond_count = 0
        splitted_suggestion = suggestion['nom_etablissement'].split(' ')
        for sp in splitted:
            if sp in splitted_suggestion:
                correspond_count += 1
        
        if correspond_count <= number_of_word - 1 and unidecode(item_to_find['locality']).lower() in unidecode(str(suggestion['adresse'])).lower():
            suggestions.append(suggestion)

    suggestions.sort()

    return suggestions

In [284]:
item = file_2.loc[0]
item

code                           NaN
id_g2a                      1733.0
name        Hôtel La Grange d'Arly
name_g2a    HOTEL LA GRANGE D'ARLY
locality                    Megève
station     MEGEVE - DEMI QUARTIER
Name: 0, dtype: object

In [285]:
suggest_simillar_word(item, name_corres)

{'id': 241, 'lieu_id': 33, 'nom_etablissement': 'RESIDENCE CGH LES GRANGES DU SOLEIL', 'adresse': 'Plagne Soleil, 73210 La Plagne-Tarentaise, France', 'loc': 222}
{'id': 595, 'lieu_id': 386, 'nom_etablissement': 'LA GRANGE DE TRAVEROLE', 'adresse': 'La Grange du Travérole, Route de Bonneval, 73480 Bessans, France', 'loc': 468}
{'id': 1377, 'lieu_id': 878, 'nom_etablissement': "RESIDENCE LES GRANGES D'ARVIEUX NOEMYS", 'adresse': 'ARVIEUX', 'loc': 1004}
{'id': 1419, 'lieu_id': 663, 'nom_etablissement': 'VILLAGE CLUB TOURISTRA LEO LAGRANGE', 'adresse': '05600 Risoul, France', 'loc': 1035}
{'id': 1733, 'lieu_id': 727, 'nom_etablissement': "HOTEL LA GRANGE D'ARLY", 'adresse': '10 Rue des Allobroges, 74120 Megève, France', 'loc': 1314}
{'id': 1967, 'lieu_id': 11, 'nom_etablissement': 'RESIDENCE LE CHALET DES GRANGES', 'adresse': nan, 'loc': 1538}
{'id': 2545, 'lieu_id': 835, 'nom_etablissement': 'VV CHALET LA GRANGE', 'adresse': nan, 'loc': 2080}
{'id': 2716, 'lieu_id': 805, 'nom_etablisseme

[{'id': 1733,
  'lieu_id': 727,
  'nom_etablissement': "HOTEL LA GRANGE D'ARLY",
  'adresse': '10 Rue des Allobroges, 74120 Megève, France',
  'loc': 1314}]

In [286]:
file_1.loc[5566]

id                                                       6284
lieu_id                                                   970
nom_etablissement                               HOTEL DU MIDI
adresse              5 place de l'hotel de ville 74230 Thones
Name: 5566, dtype: object